In [66]:
import torch
import torchvision
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
from matplotlib import pyplot as plt

In [67]:
torch.cuda.empty_cache()

In [68]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [69]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]
)

dataset = ImageFolder("train/", transform= transform)
len(dataset)

88011

In [70]:
seed = 8912
torch.manual_seed(seed)

In [71]:
batch_size = 250
train_size = 74000

val_size = len(dataset) - train_size


val_dataset, train_dataset = data.random_split(dataset, lengths=[val_size, train_size])

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, drop_last=True, num_workers=1)
val_loader = data.DataLoader(val_dataset, batch_size=100, pin_memory=True, num_workers=1)
print(len(val_dataset), len(train_dataset))
int(len(train_dataset)/batch_size)

14011 74000


296

In [72]:
class ImageClassifierNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.loss = nn.CrossEntropyLoss()

    def train_step(self, input, labels):
        input = input.to(device=device)#, non_blocking=True)
        labels = labels.to(device=device)#, non_blocking = True)

        preds = self.forward(input)
        loss = self.loss(preds, labels)

        return loss

    def val_step(self, input, labels):

        input = input.to(device=device)#, non_blocking=True)
        labels = labels.to(device=device)#, non_blocking = True)

        preds = self.forward(input)
        loss = self.loss(preds, labels)
        accuracy = self._accuracy(preds, labels)

        return {"loss":loss.detach(), "accuracy":accuracy}


    def val_epoch_end(self, preformance_measurement_data):

        accuracy = [x["accuracy"].cpu().numpy() for x in preformance_measurement_data]
        avg_accuracy = np.mean(accuracy)

        loss = [x["loss"].cpu().numpy() for x in preformance_measurement_data]
        avg_loss = np.mean(loss)

        return {"loss":{avg_loss}, "accuracy": avg_accuracy}


    def _accuracy(self, preds, labels):
        batch_size = len(preds)

        pred_indices = torch.argmax(preds, dim=1)
        return torch.tensor(torch.sum(pred_indices == labels).item() / batch_size)





In [73]:
class ResNetBlock(nn.Module):
    """ RestNet block with BN and full pre-activation """
    def __init__(self, in_channels, out_channels_conv1, out_channels_conv2, kernel_size=[3, 3], stride=[1,1], padding=[1,1]):
        super().__init__()

        self.network = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels, out_channels_conv1, kernel_size=kernel_size[0], stride=stride[0], padding=padding[0]),
            nn.ReLU(),
            nn.Conv2d(out_channels_conv1, out_channels_conv2, kernel_size=kernel_size[1], stride=stride[1], padding=padding[1])
        )


    def forward(self, input):
        return self.network(input) + input




class ResNetBlockChangeDepth(ResNetBlock):
    """ RestNet block with dimention size reduction """
    def __init__(self, in_channels, out_channels_conv1, out_channels_conv2, res_conv_kernel_size=1, res_conv_stride=1, res_conv_padding=0, kernel_size=[3, 3], stride=[1,1], padding=[1,1]):

        super().__init__(in_channels, out_channels_conv1, out_channels_conv2, kernel_size, stride, padding)

        self.conv = nn.Conv2d(in_channels, out_channels_conv2, kernel_size=res_conv_kernel_size, stride=res_conv_stride, padding=res_conv_padding)

    def forward(self, input):
        return self.network(input) + self.conv(input)



class Wrapper(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)


In [74]:
class ResNet(ImageClassifierNetwork):

    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.resnet_block1 = ResNetBlock(128, 128, 128)
        self.resnet_block2 = ResNetBlockChangeDepth(128, 256, 256)
        self.resnet_block3 = ResNetBlockChangeDepth(256, 256, 256)


        self.network = nn.Sequential(

            # Prep 3x64x64
            nn.Conv2d(in_channels, 64, kernel_size=5, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 64x32x32

            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            Wrapper(self.resnet_block1),
            nn.MaxPool2d(2, 2), # 128x16x16

            Wrapper(self.resnet_block2),
            nn.MaxPool2d(2, 2), # 256x8x8

            Wrapper(self.resnet_block3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 256x4x4

            nn.Flatten(),
            # Fully connected layer
            nn.Linear(4096, num_classes),
            # nn.Dropout(0.5),
            # nn.ReLU(),
            # nn.Linear(2048, num_classes)

        )


    def forward(self, input):
        return self.network(input)




In [75]:
model = ResNet(3, 50)
model.load_state_dict(torch.load("test_model.pt"))

model.to(device)

ResNet(
  (loss): CrossEntropyLoss()
  (resnet_block1): ResNetBlock(
    (network): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (resnet_block2): ResNetBlockChangeDepth(
    (network): Sequential(
      (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
      (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (resnet_block3): ResNetBlockChangeDepth(
    (network): Sequential(
      (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(

In [76]:
epoch = 15

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
lr_scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 1e-2, epochs=epoch, steps_per_epoch=int(len(train_dataset)/batch_size))

In [77]:
def train(train_dataloader, epoch):
    model.train(mode=True)

    #TODO Dadac agumentację dancyh

    epoch_loss = []
    batch_losses = []

    for ep in range(epoch):


        for input, label in train_dataloader:

            loss = model.train_step(input, label)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            lr_scheduler.step()

            batch_losses += [loss.item()]

        epoch_loss += [np.mean(batch_losses)]
        batch_losses = []

        print(f'Epoch {ep}; TLoss: {epoch_loss[ep]}')

        # history = []
        # for input, label in val_loader:
        #     history += [model.val_step(input, label)]

        # result = model.val_epoch_end(history)

        # print(f'Epoch {ep}; TLoss: {epoch_loss[ep]}; Vloss: {result["loss"]}; Acc: {result["accuracy"]}')


    model.eval()
    return epoch_loss

In [78]:
loss = train(train_loader, epoch)

KeyboardInterrupt: 

In [ ]:
# def eval(val_loader):
#     history = []
#     for input, label in val_loader:
#         history += [model.val_step(input, label)]

#     result = model.val_epoch_end(history)

#     return result, history

In [ ]:
# result, history = eval(val_loader)
# result

{'loss': {1.7362045}, 'accuracy': 0.55052865}

In [ ]:
# torch.save(model.state_dict(), "test_model.pt")


In [ ]:
# model = ResNet(3, 50)
# model.load_state_dict(torch.load("test_model.pt"))
